# Reconciliation #07: Ergodic vs Ensemble Divergence

## Overview
- **What this notebook tests:** Time-average and ensemble-average growth rates diverge under volatility (the core thesis of ergodic economics), and insurance narrows this gap by reducing volatility drag.
- **Prerequisites:** `pip install ergodic-insurance`
- **Estimated runtime:** < 60 seconds
- **Audience:** Developers, actuaries, and risk managers verifying the ergodic economics framework.

## Checks Performed
1. **Non-ergodicity:** Ensemble average exceeds time average in both insured and uninsured scenarios.
2. **Insurance reduces ergodic gap:** `gap(uninsured) > gap(insured)` -- insurance narrows the divergence.
3. **Insurance enhances time-average growth:** `time_avg(insured) > time_avg(uninsured)`.
4. **Statistical significance:** Bootstrap confidence interval on the growth rate difference is positive.

## Approach
We run Monte Carlo paths for two scenarios (uninsured and insured), compute ensemble-average and time-average growth rates for each, and verify the core ergodic divergence relationships. A bootstrap test confirms the growth rate improvement is statistically significant.

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reconciliation'

    if os.path.exists(os.path.join(NOTEBOOK_DIR, '_reconciliation_helpers.py')):
        print(f"Found helper module in {NOTEBOOK_DIR}")
        os.chdir(NOTEBOOK_DIR)
        if NOTEBOOK_DIR not in sys.path:
            sys.path.append(NOTEBOOK_DIR)
    else:
        print(f"WARNING: _reconciliation_helpers.py not found in {NOTEBOOK_DIR}")
        print("Please update NOTEBOOK_DIR in this cell to the correct folder path.")

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
"""Setup: imports and notebook header."""
import sys
import os
import warnings
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Reconciliation helper utilities
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df,
    create_standard_manufacturer, create_standard_loss_generator,
    create_simple_insurance_program,
)

# Core framework imports
from ergodic_insurance.simulation import Simulation, SimulationResults
from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.insurance_program import InsuranceProgram

# Display the standard notebook header
notebook_header(
    number=7,
    title="Ergodic vs Ensemble Divergence",
    description=(
        "Verifies that time-average and ensemble-average growth rates diverge under "
        "volatility, and that insurance narrows this gap by reducing volatility drag."
    ),
)

# Suppress verbose logging from the manufacturer during simulation
import logging
logging.getLogger("ergodic_insurance").setLevel(logging.WARNING)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')

# Fixed random seed for reproducibility
MASTER_SEED = 42
np.random.seed(MASTER_SEED)

print("Setup complete.")

## Section 1: Configure Scenarios

We define the manufacturer, loss model, and insurance program. Parameters are kept small
(500 paths, 20-year horizon) to target under 60 seconds of runtime while still providing
enough statistical power to detect the ergodic divergence.

In [ ]:
"""Configure simulation parameters for both scenarios."""
section_header("1. Configure Scenarios")

# Simulation parameters
N_PATHS = 500
TIME_HORIZON = 20

# Manufacturer parameters
INITIAL_ASSETS = 10_000_000
ASSET_TURNOVER = 1.2
OPERATING_MARGIN = 0.10
TAX_RATE = 0.25
RETENTION_RATIO = 0.70

# Loss parameters -- moderate frequency and severity to create
# meaningful volatility drag without excessive insolvency
LOSS_FREQUENCY = 0.15
LOSS_SEVERITY_MEAN = 500_000
LOSS_SEVERITY_STD = 600_000

# Insurance parameters
DEDUCTIBLE = 50_000
LIMIT = 2_000_000
RATE = 0.03

# Create the insurance program
insurance_program = create_simple_insurance_program(
    deductible=DEDUCTIBLE,
    limit=LIMIT,
    rate=RATE,
)

annual_premium = insurance_program.calculate_premium()
expected_annual_loss = LOSS_FREQUENCY * LOSS_SEVERITY_MEAN
ebit = INITIAL_ASSETS * ASSET_TURNOVER * OPERATING_MARGIN

print(f"Simulation Configuration")
print(f"  Paths:        {N_PATHS}")
print(f"  Horizon:      {TIME_HORIZON} years")
print(f"  Master seed:  {MASTER_SEED}")
print()
print(f"Manufacturer")
print(f"  Initial assets:      {fmt_dollar(INITIAL_ASSETS)}")
print(f"  Annual revenue:      {fmt_dollar(INITIAL_ASSETS * ASSET_TURNOVER)}")
print(f"  EBIT:                {fmt_dollar(ebit)}")
print(f"  Expected ann. loss:  {fmt_dollar(expected_annual_loss)}")
print()
print(f"Insurance Program")
print(f"  Deductible:          {fmt_dollar(DEDUCTIBLE)}")
print(f"  Limit:               {fmt_dollar(LIMIT)}")
print(f"  Annual premium:      {fmt_dollar(annual_premium)}")
print(f"  Premium / E[Loss]:   {annual_premium / expected_annual_loss:.2f}x")

## Section 2: Run Uninsured Simulations

Run `N_PATHS` independent simulation paths with no insurance coverage.
Each path uses a different seed for its loss generator to produce independent trajectories.

In [ ]:
"""Run uninsured Monte Carlo paths."""
section_header("2. Run Uninsured Simulations")

with timed_cell("Uninsured simulations"):
    uninsured_results = []
    for i in range(N_PATHS):
        seed_i = MASTER_SEED + i
        mfr = create_standard_manufacturer(
            initial_assets=INITIAL_ASSETS,
            asset_turnover=ASSET_TURNOVER,
            operating_margin=OPERATING_MARGIN,
            tax_rate=TAX_RATE,
            retention_ratio=RETENTION_RATIO,
        )
        loss_gen = ManufacturingLossGenerator.create_simple(
            seed=seed_i,
            frequency=LOSS_FREQUENCY,
            severity_mean=LOSS_SEVERITY_MEAN,
            severity_std=LOSS_SEVERITY_STD,
        )
        sim = Simulation(
            manufacturer=mfr,
            loss_generator=loss_gen,
            insurance_policy=None,
            time_horizon=TIME_HORIZON,
            seed=seed_i,
        )
        uninsured_results.append(sim.run())

n_survived_uninsured = sum(
    1 for r in uninsured_results if r.insolvency_year is None and r.equity[-1] > 0
)
print(f"Uninsured: {N_PATHS} paths completed")
print(f"  Survived: {n_survived_uninsured}/{N_PATHS} ({n_survived_uninsured/N_PATHS*100:.1f}%)")

## Section 3: Run Insured Simulations

Run `N_PATHS` independent paths with the same loss seeds but with insurance coverage applied.
Using the same seeds allows paired comparison -- each insured path faces identical losses
as the corresponding uninsured path.

In [ ]:
"""Run insured Monte Carlo paths with same loss seeds."""
section_header("3. Run Insured Simulations")

with timed_cell("Insured simulations"):
    insured_results = []
    for i in range(N_PATHS):
        seed_i = MASTER_SEED + i
        mfr = create_standard_manufacturer(
            initial_assets=INITIAL_ASSETS,
            asset_turnover=ASSET_TURNOVER,
            operating_margin=OPERATING_MARGIN,
            tax_rate=TAX_RATE,
            retention_ratio=RETENTION_RATIO,
        )
        loss_gen = ManufacturingLossGenerator.create_simple(
            seed=seed_i,
            frequency=LOSS_FREQUENCY,
            severity_mean=LOSS_SEVERITY_MEAN,
            severity_std=LOSS_SEVERITY_STD,
        )
        sim = Simulation(
            manufacturer=mfr,
            loss_generator=loss_gen,
            insurance_policy=insurance_program,
            time_horizon=TIME_HORIZON,
            seed=seed_i,
        )
        insured_results.append(sim.run())

n_survived_insured = sum(
    1 for r in insured_results if r.insolvency_year is None and r.equity[-1] > 0
)
print(f"Insured: {N_PATHS} paths completed")
print(f"  Survived: {n_survived_insured}/{N_PATHS} ({n_survived_insured/N_PATHS*100:.1f}%)")

## Section 4: Calculate Growth Rates

For each scenario we compute:
- **Ensemble average growth:** `mean(final / initial)` across all paths (arithmetic mean of growth factors).
- **Time average growth:** `exp(mean(log(final / initial)))` -- the geometric mean, representing the growth rate a single entity actually experiences over time.
- **Ergodic gap:** The difference between ensemble and time averages.

In [ ]:
"""Calculate ensemble-average and time-average growth rates for both scenarios."""
section_header("4. Calculate Growth Rates")

analyzer = ErgodicAnalyzer()

# ---------- Uninsured ----------
# Time-average: calculate per-path growth rate, then average
uninsured_ta_rates = []
for r in uninsured_results:
    g = analyzer.calculate_time_average_growth(r.equity)
    if np.isfinite(g):
        uninsured_ta_rates.append(g)

uninsured_ta_mean = float(np.mean(uninsured_ta_rates)) if uninsured_ta_rates else -np.inf

# Ensemble average: mean of growth rates across paths
uninsured_trajectories = [r.equity for r in uninsured_results]
uninsured_ensemble_stats = analyzer.calculate_ensemble_average(
    uninsured_trajectories, metric="growth_rate"
)
uninsured_ea_mean = uninsured_ensemble_stats["mean"]

# Ergodic gap
uninsured_gap = uninsured_ea_mean - uninsured_ta_mean

# ---------- Insured ----------
insured_ta_rates = []
for r in insured_results:
    g = analyzer.calculate_time_average_growth(r.equity)
    if np.isfinite(g):
        insured_ta_rates.append(g)

insured_ta_mean = float(np.mean(insured_ta_rates)) if insured_ta_rates else -np.inf

insured_trajectories = [r.equity for r in insured_results]
insured_ensemble_stats = analyzer.calculate_ensemble_average(
    insured_trajectories, metric="growth_rate"
)
insured_ea_mean = insured_ensemble_stats["mean"]

insured_gap = insured_ea_mean - insured_ta_mean

# ---------- Display ----------
summary_data = {
    "Metric": [
        "Ensemble Average Growth Rate",
        "Time Average Growth Rate",
        "Ergodic Gap (EA - TA)",
        "Surviving Paths",
        "Valid TA Paths",
    ],
    "Uninsured": [
        f"{uninsured_ea_mean*100:.4f}%",
        f"{uninsured_ta_mean*100:.4f}%",
        f"{uninsured_gap*100:.4f}%",
        f"{n_survived_uninsured}/{N_PATHS}",
        f"{len(uninsured_ta_rates)}/{N_PATHS}",
    ],
    "Insured": [
        f"{insured_ea_mean*100:.4f}%",
        f"{insured_ta_mean*100:.4f}%",
        f"{insured_gap*100:.4f}%",
        f"{n_survived_insured}/{N_PATHS}",
        f"{len(insured_ta_rates)}/{N_PATHS}",
    ],
}
df_summary = pd.DataFrame(summary_data)
display_df(df_summary, title="Growth Rate Comparison")

print(f"\nKey Results:")
print(f"  Uninsured ergodic gap: {uninsured_gap*100:.4f}%")
print(f"  Insured ergodic gap:   {insured_gap*100:.4f}%")
print(f"  Gap reduction:         {(uninsured_gap - insured_gap)*100:.4f}%")
print(f"  TA growth improvement: {(insured_ta_mean - uninsured_ta_mean)*100:.4f}%")

## Section 5: Ergodic Gap Analysis and Reconciliation Checks

Now we verify the core assertions of the ergodic economics framework:
1. Non-ergodicity: ensemble average > time average for both scenarios
2. Insurance reduces the gap
3. Insurance enhances time-average growth

In [ ]:
"""Verify ergodic divergence properties."""
section_header("5. Ergodic Gap Analysis")

checker_ergodic = ReconciliationChecker(section="Ergodic Divergence Properties")

# Check 1: Non-ergodicity -- ensemble average > time average (uninsured)
checker_ergodic.assert_greater(
    uninsured_ea_mean, uninsured_ta_mean,
    message="Uninsured: ensemble avg > time avg (non-ergodicity)",
)

# Check 2: Non-ergodicity -- ensemble average > time average (insured)
checker_ergodic.assert_greater(
    insured_ea_mean, insured_ta_mean,
    message="Insured: ensemble avg > time avg (non-ergodicity)",
)

# Check 3: Insurance reduces the ergodic gap
checker_ergodic.assert_greater(
    uninsured_gap, insured_gap,
    message="Ergodic gap (uninsured) > ergodic gap (insured)",
)

# Check 4: Insurance enhances time-average growth
checker_ergodic.assert_greater(
    insured_ta_mean, uninsured_ta_mean,
    message="Time-avg growth (insured) > time-avg growth (uninsured)",
)

# Check 5: Both ergodic gaps are positive (fundamental requirement)
checker_ergodic.assert_greater(
    uninsured_gap, 0.0,
    message="Uninsured ergodic gap is positive",
)
checker_ergodic.assert_greater(
    insured_gap, 0.0,
    message="Insured ergodic gap is positive",
)

checker_ergodic.display_results()

## Section 6: Growth Trajectory Visualization

Visualize the divergence between insured and uninsured scenarios:
- Sample equity paths for both scenarios
- Growth rate distributions
- Ensemble vs time average evolution
- Confidence bands

In [ ]:
"""Visualize growth trajectories and ergodic divergence."""
section_header("6. Growth Trajectory Visualization")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- Panel 1: Sample equity paths ---
ax = axes[0, 0]
n_sample = min(30, N_PATHS)
for i in range(n_sample):
    if insured_results[i].equity[-1] > 0:
        ax.plot(insured_results[i].years, insured_results[i].equity,
                alpha=0.2, color='#2196F3', lw=0.5)
    if uninsured_results[i].equity[-1] > 0:
        ax.plot(uninsured_results[i].years, uninsured_results[i].equity,
                alpha=0.2, color='#F44336', lw=0.5)
ax.set_xlabel('Year')
ax.set_ylabel('Equity ($)')
ax.set_yscale('log')
ax.set_title('Sample Equity Paths (Blue=Insured, Red=Uninsured)')
ax.grid(True, alpha=0.3)

# --- Panel 2: Time-average growth rate distributions ---
ax = axes[0, 1]
if insured_ta_rates:
    ax.hist(np.array(insured_ta_rates) * 100, bins=40, alpha=0.5,
            label=f'Insured (median={np.median(insured_ta_rates)*100:.2f}%)',
            color='#2196F3', density=True)
    ax.axvline(np.median(insured_ta_rates) * 100, color='#1565C0',
               ls='--', lw=2, label='Insured median')
if uninsured_ta_rates:
    ax.hist(np.array(uninsured_ta_rates) * 100, bins=40, alpha=0.5,
            label=f'Uninsured (median={np.median(uninsured_ta_rates)*100:.2f}%)',
            color='#F44336', density=True)
    ax.axvline(np.median(uninsured_ta_rates) * 100, color='#B71C1C',
               ls='--', lw=2, label='Uninsured median')
ax.set_xlabel('Time-Average Growth Rate (%/yr)')
ax.set_ylabel('Density')
ax.set_title('Time-Average Growth Rate Distribution')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# --- Panel 3: Ensemble mean vs time-average mean over horizon ---
ax = axes[1, 0]
# Compute running ensemble mean of equity and time-average approximation
years = np.arange(TIME_HORIZON)

# Build equity arrays for surviving paths
insured_equity_matrix = np.array([r.equity for r in insured_results])
uninsured_equity_matrix = np.array([r.equity for r in uninsured_results])

# Ensemble mean at each time point
ins_ensemble_curve = np.mean(insured_equity_matrix, axis=0)
unins_ensemble_curve = np.mean(uninsured_equity_matrix, axis=0)

# Geometric mean (time-average proxy) at each time point
# Use only positive values to avoid log domain errors
def geometric_mean_curve(matrix):
    """Compute geometric mean across paths at each time step."""
    result = np.zeros(matrix.shape[1])
    for t in range(matrix.shape[1]):
        vals = matrix[:, t]
        positive_vals = vals[vals > 0]
        if len(positive_vals) > 0:
            result[t] = np.exp(np.mean(np.log(positive_vals)))
    return result

ins_geomean_curve = geometric_mean_curve(insured_equity_matrix)
unins_geomean_curve = geometric_mean_curve(uninsured_equity_matrix)

ax.plot(years, ins_ensemble_curve / 1e6, color='#2196F3', lw=2,
        label='Insured (ensemble mean)')
ax.plot(years, ins_geomean_curve / 1e6, color='#2196F3', lw=2, ls='--',
        label='Insured (geometric mean)')
ax.plot(years, unins_ensemble_curve / 1e6, color='#F44336', lw=2,
        label='Uninsured (ensemble mean)')
ax.plot(years, unins_geomean_curve / 1e6, color='#F44336', lw=2, ls='--',
        label='Uninsured (geometric mean)')
ax.set_xlabel('Year')
ax.set_ylabel('Equity ($M)')
ax.set_title('Ensemble Mean vs Geometric Mean (Ergodic Divergence)')
ax.legend(fontsize=7)
ax.grid(True, alpha=0.3)

# --- Panel 4: Ergodic gap bar chart ---
ax = axes[1, 1]
scenarios = ['Uninsured', 'Insured']
ea_values = [uninsured_ea_mean * 100, insured_ea_mean * 100]
ta_values = [uninsured_ta_mean * 100, insured_ta_mean * 100]
gap_values = [uninsured_gap * 100, insured_gap * 100]

x = np.arange(len(scenarios))
width = 0.25
bars_ea = ax.bar(x - width, ea_values, width, label='Ensemble Avg', color='#FFB74D')
bars_ta = ax.bar(x, ta_values, width, label='Time Avg', color='#4CAF50')
bars_gap = ax.bar(x + width, gap_values, width, label='Ergodic Gap', color='#9C27B0')
ax.set_xticks(x)
ax.set_xticklabels(scenarios)
ax.set_ylabel('Growth Rate (%/yr)')
ax.set_title('Growth Rates and Ergodic Gap')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Visualization complete.")

## Section 7: Statistical Significance

We use two approaches to assess significance:
1. **Welch's t-test** on the per-path time-average growth rates (insured vs uninsured)
2. **Bootstrap confidence interval** on the mean growth rate difference

In [ ]:
"""Statistical significance testing."""
section_header("7. Statistical Significance")

checker_significance = ReconciliationChecker(section="Statistical Significance")

with timed_cell("Significance tests"):
    # Welch's t-test
    if insured_ta_rates and uninsured_ta_rates:
        t_stat, p_value = analyzer.significance_test(
            insured_ta_rates, uninsured_ta_rates, test_type="greater"
        )
        print(f"Welch's t-test (insured > uninsured):")
        print(f"  t-statistic: {t_stat:.4f}")
        print(f"  p-value:     {p_value:.6f}")
        print(f"  Significant at 5%: {'Yes' if p_value < 0.05 else 'No'}")

        checker_significance.check(
            p_value < 0.05,
            message="Welch's t-test: insured TA growth > uninsured (p < 0.05)",
            detail=f"t={t_stat:.4f}, p={p_value:.6f}",
        )
    else:
        print("Insufficient data for t-test.")
        checker_significance.check(False, "Sufficient data for t-test")

    # Bootstrap confidence interval on the mean difference
    N_BOOTSTRAP = 10_000
    rng = np.random.RandomState(MASTER_SEED)

    ins_arr = np.array(insured_ta_rates)
    unins_arr = np.array(uninsured_ta_rates)

    boot_diffs = np.zeros(N_BOOTSTRAP)
    for b in range(N_BOOTSTRAP):
        ins_sample = rng.choice(ins_arr, size=len(ins_arr), replace=True)
        unins_sample = rng.choice(unins_arr, size=len(unins_arr), replace=True)
        boot_diffs[b] = np.mean(ins_sample) - np.mean(unins_sample)

    ci_lower = np.percentile(boot_diffs, 2.5)
    ci_upper = np.percentile(boot_diffs, 97.5)
    boot_mean = np.mean(boot_diffs)

    print(f"\nBootstrap CI on growth rate difference (insured - uninsured):")
    print(f"  Mean difference:  {boot_mean*100:.4f}%")
    print(f"  95% CI:           [{ci_lower*100:.4f}%, {ci_upper*100:.4f}%]")
    print(f"  CI excludes zero: {'Yes' if ci_lower > 0 else 'No'}")

    # Check: bootstrap CI lower bound > 0 (insurance truly helps)
    checker_significance.check(
        ci_lower > 0,
        message="Bootstrap 95% CI lower bound > 0 (growth improvement is real)",
        detail=f"CI=[{ci_lower*100:.4f}%, {ci_upper*100:.4f}%]",
    )

    # Check: bootstrap mean is consistent with direct calculation
    direct_diff = insured_ta_mean - uninsured_ta_mean
    checker_significance.assert_close(
        boot_mean, direct_diff, tol=0.005,
        message="Bootstrap mean ~ direct difference (consistency)",
        label_actual="Bootstrap mean", label_expected="Direct diff",
    )

checker_significance.display_results()

## Section 8: Cross-Validation with ErgodicAnalyzer.compare_scenarios

Verify that our manual calculations agree with the framework's built-in
`compare_scenarios` method, which performs the same ergodic analysis internally.

In [ ]:
"""Cross-validate against ErgodicAnalyzer.compare_scenarios."""
section_header("8. Cross-Validation with compare_scenarios()")

checker_crossval = ReconciliationChecker(section="Cross-Validation")

with timed_cell("Cross-validation"):
    import warnings as _warnings
    with _warnings.catch_warnings():
        _warnings.simplefilter("ignore", DeprecationWarning)

        comparison = analyzer.compare_scenarios(
            insured_results=insured_results,
            uninsured_results=uninsured_results,
            metric="equity",
        )

        # Extract metrics using attribute access
        fw_ins_ta = comparison.insured.time_average_mean
        fw_unins_ta = comparison.uninsured.time_average_mean
        fw_ins_ea = comparison.insured.ensemble_average
        fw_unins_ea = comparison.uninsured.ensemble_average
        fw_ta_gain = comparison.ergodic_advantage.time_average_gain

    print(f"Framework compare_scenarios results:")
    print(f"  Insured TA mean:   {fw_ins_ta*100:.4f}%")
    print(f"  Uninsured TA mean: {fw_unins_ta*100:.4f}%")
    print(f"  TA gain:           {fw_ta_gain*100:.4f}%")
    print(f"  Insured EA:        {fw_ins_ea*100:.4f}%")
    print(f"  Uninsured EA:      {fw_unins_ea*100:.4f}%")

    # Check: framework time-average matches our manual calculation
    checker_crossval.assert_close(
        fw_ins_ta, insured_ta_mean, tol=1e-6,
        message="Framework insured TA ~ manual insured TA",
        label_actual="Framework", label_expected="Manual",
    )
    checker_crossval.assert_close(
        fw_unins_ta, uninsured_ta_mean, tol=1e-6,
        message="Framework uninsured TA ~ manual uninsured TA",
        label_actual="Framework", label_expected="Manual",
    )

    # Check: framework ensemble average matches our calculation
    checker_crossval.assert_close(
        fw_ins_ea, insured_ea_mean, tol=1e-6,
        message="Framework insured EA ~ manual insured EA",
        label_actual="Framework", label_expected="Manual",
    )
    checker_crossval.assert_close(
        fw_unins_ea, uninsured_ea_mean, tol=1e-6,
        message="Framework uninsured EA ~ manual uninsured EA",
        label_actual="Framework", label_expected="Manual",
    )

    # Check: TA gain is positive (framework agrees insurance helps)
    checker_crossval.assert_greater(
        fw_ta_gain, 0.0,
        message="Framework reports positive TA growth gain from insurance",
    )

checker_crossval.display_results()

## Section 9: Combined Summary

A consolidated view of all reconciliation checks performed in this notebook.

In [ ]:
"""Combined summary of all checks."""
section_header("9. Combined Summary")

sections = [
    ("Ergodic Divergence Properties", checker_ergodic),
    ("Statistical Significance", checker_significance),
    ("Cross-Validation", checker_crossval),
]

summary_rows = []
for name, checker in sections:
    passed, failed = checker.summary_counts
    total = passed + failed
    summary_rows.append({
        "Check Section": name,
        "Passed": passed,
        "Failed": failed,
        "Total": total,
        "Status": "ALL PASS" if failed == 0 else f"{failed} FAILED",
    })

df_combined = pd.DataFrame(summary_rows)
display_df(df_combined, title="Reconciliation Check Summary")

total_passed = sum(r["Passed"] for r in summary_rows)
total_failed = sum(r["Failed"] for r in summary_rows)
total_checks = total_passed + total_failed
print(f"\nTotal: {total_passed}/{total_checks} checks passed")
if total_failed == 0:
    print("All ergodic divergence checks PASSED.")
else:
    print(f"WARNING: {total_failed} check(s) FAILED.")

## Final Result

The cell below produces the final PASS/FAIL banner. If any check failed,
it raises an `AssertionError` so that `nbconvert` or CI will catch the failure.

In [ ]:
"""Final PASS/FAIL summary -- raises AssertionError on failure."""
final_summary(checker_ergodic, checker_significance, checker_crossval)